# DB2-Salesforce connector: Basic user information updates

In [1]:
# Parameters
hours_range = 24*365*30 # number of hours to look back

# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# login parameters to be handled by Papermill
sf_login_params = None
sql_login_params = None

lib_dir = ''

In [2]:
# Parameters
from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')
wang159_myrmekes_db = application.new_db_engine('wang159_myrmekes')


salesforce = application.new_salesforce_engine()
db_s = salesforce


nanoHUB - Serving Students, Researchers & Instructors
Obtained Salesforce access token ...... True


In [3]:
import sys
sys.path.append(lib_dir)

import pandas as pd
import datetime

## Obtain contacts from DB2
that have last visit date within range of interest specified by day_range

In [4]:
# Depending on the task, use different query and cutoff

# Hourly update for new registrations
date_cutoff = (datetime.datetime.today().date() - datetime.timedelta(hours=hours_range)).strftime('%Y-%m-%d')
sql_query = "select id, name, username, block, email, sendEmail, registerDate, lastvisitDate \
                    from jos_users where registerDate >= '%s'" % date_cutoff

# display
print(sql_query)

select id, name, username, block, email, sendEmail, registerDate, lastvisitDate                     from jos_users where registerDate >= '1991-11-20'


In [5]:
# connect with DB2
import sqlalchemy as sql

df = pd.read_sql_query(sql_query, nanohub_db)

# get user profile details
profile_df = pd.read_sql_query("select * from jos_user_profiles where profile_key in ('orgtype', 'organization', 'orcid')", nanohub_db)

In [6]:
# display
display(df.head())
display(df.tail())

display(profile_df.head(1))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate
0,998,hubrepo hubrepo,hubrepo,0,nkissebe@gmail.com,0,2014-11-13 21:09:09,NaT
1,1683,nanoHUB support,support,0,support@nanohub.org,0,2008-11-19 22:51:04,2008-11-19 23:55:30
2,1684,Grid Statistics,gridstat,0,gridstat@nanohub.org,0,2008-11-18 17:29:56,2020-02-14 18:50:14
3,1685,NCN NCN,ncn,0,ncn@nanohub.org,0,2008-11-11 19:17:04,NaT
4,1686,nanoHUB nanoHUB,nanohub,0,apps@nanohub.org,0,2014-06-26 19:38:57,NaT


,id,name,username,block,email,sendEmail,registerDate,lastvisitDate
263937,345791,VENKATA ANIRUDH KOMAL KODAVALI,anirudh_k,0,venkata_a@ec.iitr.ac.in,0,2021-11-12 18:06:19,2021-11-12 18:06:19
263938,345792,vishesh chandrikapure,visheshc38,0,visheshc38@gmail.com,0,2021-11-12 18:15:28,2021-11-12 18:15:28
263939,345793,Rahul Kumar,rahul20150133775,0,rahul20150133775@gmail.com,0,2021-11-12 18:30:07,2021-11-12 18:30:07
263940,345795,Karan Kannoujiya,kannoujiya432,0,kannoujiya432@gmail.com,0,2021-11-12 18:37:49,2021-11-12 18:37:49
263941,345796,sarana solusindo informatika,solusitraining,0,annurulizza3@gmail.com,0,2021-11-12 18:42:07,2021-11-12 18:43:58


,id,user_id,profile_key,profile_value,ordering,access
0,1,15623,orgtype,university,2,5


In [7]:
df = pd.merge(df, profile_df[profile_df['profile_key'] == 'orgtype'][['user_id', 'profile_value']],how='left', left_on='id', right_on='user_id').drop(columns=['user_id']).rename(columns={'profile_value':'orgtype'})

df = pd.merge(df, profile_df[profile_df['profile_key'] == 'organization'][['user_id', 'profile_value']],how='left', left_on='id', right_on='user_id').drop(columns=['user_id']).rename(columns={'profile_value':'organization'})

df = pd.merge(df, profile_df[profile_df['profile_key'] == 'orcid'][['user_id', 'profile_value']],how='left', left_on='id', right_on='user_id').drop(columns=['user_id']).rename(columns={'profile_value':'orcid'})

# display
display(df.head(1))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid
0,998,hubrepo hubrepo,hubrepo,0,nkissebe@gmail.com,0,2014-11-13 21:09:09,NaT,NaN,NaN,NaN


In [8]:
# Obtain company domain information from DB2
domain_df = pd.read_sql_query("select name as domain_name, domain, industry, `size range` as size, country from wang159_myrmekes.companies_email_domain", nanohub_db)

display(domain_df.head())

,domain_name,domain,industry,size,country
0,strategic interns,strategicinterns.com,education management,1-10,india
1,bosque y comunidad,bosqueycomunidad.org,non-profit organization management,1-10,spain
2,alaric as,alaric.no,management consulting,1-10,None
3,tws technology inc.,talkwithsam.co,computer software,1-10,None
4,"cardinal strategies, llc",cardinal-strategies.com,civil engineering,11-50,united states


In [9]:
# make sure domain is unique and drop NaN
domain_df = domain_df.drop_duplicates(subset='domain', keep='first').dropna()

In [10]:
# get email domain of nanoHUB users
def get_domain(this_email):
    seg_list = this_email.split('@')

    if len(seg_list) == 2:
        return seg_list[1].lower()
    else:
        return None

df['email_host'] = df['email'].apply(get_domain)

In [11]:
# get domain by nanoHUB organization field
df['organization'] = df.organization.astype('str').apply(str.lower)

df.loc[(df['organization']=='nan'),'organization'] = None

In [12]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
s_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/saxenap/venv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
import re

def clean_domain(this_domain_name):

    if not this_domain_name:
        return None

    # Replace all non-alphanumeric characters with space
    this_domain_name = re.sub("[^0-9a-zA-Z]+", " ", this_domain_name.lower())
    name_list = this_domain_name.split(' ')

    # remove all stop words
    name_list = ['' if x in s_words else x for x in name_list]

    return set(filter(None, name_list))

In [14]:
# clean domain name
domain_df['domain_cleaned_set'] = domain_df.domain_name.apply(clean_domain)

# clean nanohub org name
df['org_cleaned_set'] = df.organization.apply(clean_domain)

In [15]:
# hash cleaned domain name
domain_df['domain_cleaned_hash'] = domain_df['domain_cleaned_set'].apply(lambda x: '-'.join(sorted(list(x))) if x else None)

# hash cleaned clean nanohub org name
df['org_cleaned_hash'] = df['org_cleaned_set'].apply(lambda x: '-'.join(sorted(list(x))) if x else None)

## Collaborative filtering for commerical email hosts

In [16]:
def predict_if_commerical(this_email_host_df):
    # predict if this email host is commerical
    top_cnt = this_email_host_df['org_cleaned_hash'].value_counts()

    if top_cnt.shape[0] > 0:
        if (top_cnt.iloc[0]/top_cnt.sum()) < 0.2:
            return True
        else:
            return False

    else:
        return False

is_email_commerical = df[['email_host', 'org_cleaned_hash']].groupby('email_host').apply(predict_if_commerical)

In [17]:
# mark commerical email addresses
df = pd.merge(df, is_email_commerical.reset_index(name='is_email_commerical'), how='left', left_on='email_host', right_on='email_host' ,)

In [18]:
# obtain domain estimation from email addresses
df = pd.merge(df, domain_df[~domain_df.domain.isin(is_email_commerical[is_email_commerical].index)]['domain'], how='left', left_on='email_host', right_on='domain').rename(columns={'domain':'domain_by_email'})

# display
display(df.head(2).T)

,0,1
id,998,1683
name,hubrepo hubrepo,nanoHUB support
username,hubrepo,support
block,0,0
email,nkissebe@gmail.com,support@nanohub.org
sendEmail,0,0
registerDate,2014-11-13 21:09:09,2008-11-19 22:51:04
lastvisitDate,NaT,2008-11-19 23:55:30
orgtype,NaN,university
organization,None,purdue university


## Fuzzy derive organization from nanoHUB profiles

In [19]:
# attempt direct join by hash
domain_subset_df = domain_df[domain_df.domain.isin(df.email_host.unique())][['domain', 'domain_cleaned_hash']]
domain_subset_df = domain_subset_df[~domain_subset_df.domain.isna()&~domain_subset_df.domain_cleaned_hash.isna()]

derived_df = pd.merge(df, domain_subset_df.rename(columns={'domain':'domain_by_profile'}),how='left', left_on='org_cleaned_hash', right_on='domain_cleaned_hash').drop('domain_cleaned_hash', axis=1)

# display
derived_df.sample(5).T

,138129,34372,143786,135674,233466
id,161580,37776,169032,158575,306619
name,Yunqing Tang,Max Paganini,Christopher Sole,Yong Zhang,Daniel sousa Filho
username,yunqing,mpaganin,cgsole46,rafemicky,danielsousaflor2014
block,0,0,0,0,0
email,yqing518@gmail.com,mpaganin@purdue.edu,cgsole46@liv.ac.uk,hitrafe@163.com,danielsousaflor2014@gmail.com
sendEmail,0,0,0,0,0
registerDate,2017-01-09 10:37:54,2009-09-03 18:06:48,2017-04-07 15:39:31,2016-11-21 09:10:25,2020-11-08 15:32:13
lastvisitDate,2017-02-01 19:04:42,2009-09-04 01:30:46,2017-04-07 15:40:05,2016-11-21 09:10:40,2020-12-02 12:04:19
orgtype,NaN,universityundergraduate,NaN,NaN,universityfaculty
organization,None,purdue university,None,None,None


In [20]:
# attempt to reduce domain_df size to speedup Jaccard calculation
from itertools import chain

all_word_set = set(chain.from_iterable(derived_df[(derived_df['domain_by_profile'].isna() & (~derived_df['organization'].isna()))].org_cleaned_set.values))

# select the domain_df entry with words from all_word_set
domain_subset_df = domain_df[domain_df.domain_cleaned_set.notnull()]
domain_subset_df = domain_subset_df[domain_subset_df.domain_cleaned_set.apply(lambda x: True if len(x.intersection(all_word_set)) > 0 else False)]

In [21]:
def get_org(this_df):
    # look at the cohort with this label, do they have a commonly agreed domain_by_email? If so, use it
    this_dbe_count = this_df.domain_by_email.value_counts()

    if this_dbe_count.shape[0] < 1:
        return None

    this_top_dbe_perc = this_dbe_count.iloc[0]/this_dbe_count.sum()

    # if top choice occupies majority of cases, then this cohort can be implied to this domain_by_email address
    if this_top_dbe_perc > 0.5:
        return this_dbe_count.index[0]


derived_hash = derived_df.groupby('org_cleaned_hash').apply(get_org)

In [22]:
derived_df = pd.merge(derived_df, derived_hash.reset_index(name='domain_by_profile_infer'), how='left', left_on='org_cleaned_hash', right_on='org_cleaned_hash')

In [23]:
# calculate a composite domain based on domain_by_email, domain_by_profile, and domain_by_profile_infer
def get_composite_domain(this_df):

    # if domain found via profile with direct match, use it
    if this_df.domain_by_profile != '':
        return this_df.domain_by_profile

    # if domain found via email, use it
    if this_df.domain_by_email != '':
        return this_df.domain_by_email

        # if domain found via profile but not with a direct match, use it
    if this_df.domain_by_profile_infer != '':
        return this_df.domain_by_profile_infer

    # else, no organization can be concluded, return none
    return None


derived_df['domain_final'] = derived_df[['domain_by_email', 'domain_by_profile', 'domain_by_profile_infer']].fillna('').apply(get_composite_domain, axis=1)

In [24]:
derived_df[derived_df.email_host == 'yahoo.com'][['domain_final', 'domain_by_email']]

,domain_final,domain_by_email
8,None,None
17,None,None
42,None,None
52,None,None
76,None,None
...,...,...
275120,utah.edu,NaN
275572,None,NaN
275717,bsu.edu,NaN
275985,iupui.edu,NaN


## Update Salesforce institution

In [25]:
# get a unique list of organizations that appears in contact
#org_df = derived_df[['domain_name', 'domain', 'industry', 'size', 'country']].drop_duplicates().dropna()
org_df = pd.merge(derived_df[['domain_final']].drop_duplicates().dropna(), domain_df, how='inner', left_on='domain_final', right_on='domain')
org_df = org_df[['domain_name', 'domain', 'industry', 'size', 'country']]

# display
display(org_df.head(2))

,domain_name,domain,industry,size,country
0,purdue university,purdue.edu,higher education,10001+,united states
1,netzero inc,netzero.net,computer software,1-10,united states


In [26]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Name'] = org_df['domain_name']

df_sf['Size_Range__c'] = org_df['size'].fillna('Unknown')
df_sf['Country__c'] = org_df['country']
df_sf['Domain__c'] = org_df['domain']
df_sf['Industry__c'] = org_df['industry']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Name,purdue university,netzero inc
Size_Range__c,10001+,1-10
Country__c,united states,united states
Domain__c,purdue.edu,netzero.net
Industry__c,higher education,computer software


In [27]:


db_s.object_id = 'organization__c'
db_s.external_id = 'Domain__c'

In [28]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000b4jKRAAY
hello
[Success] CSV upload successful. Job ID = 7505w00000b4jKRAAY
[Success] Closing job successful. Job ID = 7505w00000b4jKRAAY


In [29]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000b4jKRAAY',
 'operation': 'upsert',
 'object': 'organization__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-11-12T18:53:30.000+0000',
 'systemModstamp': '2021-11-12T18:53:34.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'Domain__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [30]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''


## Obtain newly updated organization IDs from Salesforce

In [31]:
# create DB2 to Salesforce API object


In [32]:
# get Salesforce ID for organizations
sf_org_ID_df = db_s.query_data('SELECT Id, Domain__c FROM organization__c')

# display
sf_org_ID_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000b4jKqAAI
{"id":"7505w00000b4jKqAAI","operation":"query","object":"organization__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-11-12T18:53:38.000+0000","systemModstamp":"2021-11-12T18:53:41.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":6485,"retries":0,"totalProcessingTime":406}
[Success] Bulk job completed successfully.


,Domain__c,Id
0,upenn.edu,a0r5w00000V42c0AAB
1,fer.hr,a0r5w00000V42c1AAB
2,wheatoncollege.edu,a0r5w00000V42c2AAB


In [33]:
# join salesforce ID back to contact DF
df = pd.merge(sf_org_ID_df, derived_df, how='right', left_on='Domain__c', right_on='domain_final').drop('Domain__c', axis=1).rename(columns={'Id':'Salesforce_org_ID'})

In [34]:
# display
display(df.head(2))

,Salesforce_org_ID,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash,is_email_commerical,domain_by_email,domain_by_profile,domain_by_profile_infer,domain_final
0,NaN,998,hubrepo hubrepo,hubrepo,0,nkissebe@gmail.com,0,2014-11-13 21:09:09,NaT,NaN,None,NaN,gmail.com,None,None,True,NaN,NaN,NaN,None
1,a0r5w00000V42cCAAR,1683,nanoHUB support,support,0,support@nanohub.org,0,2008-11-19 22:51:04,2008-11-19 23:55:30,university,purdue university,NaN,nanohub.org,"{university, purdue}",purdue-university,False,NaN,purdue.edu,purdue.edu,purdue.edu


## Match data with Salesforce format

In [35]:
# split full name into first, middle, and last names
def split_full_name(this_name):
    try:
        this_name_list = list(filter(None, this_name.split(' ')))
        if this_name_list is not None and type(this_name_list) is list:
            if len(this_name_list) == 1:
                # single word name
                return pd.Series([this_name_list[0],None, this_name_list[0]])
            
            elif len(this_name_list) > 1:
                # multi word name
                return pd.Series([this_name_list[0],' '.join(this_name_list[1:-1]),this_name_list[-1]])
            
            else:
                return pd.Series([None, None, None])

    except Exception as e:
        print(e)
        raise e
        

In [36]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df['name'] = df.name.str.strip()
df = df[df['name'].notnull()]

display(df)
df_sf[['firstname', 'Middle_name__c', 'lastname']] = df['name'].apply(split_full_name)

display(df_sf[(df_sf['firstname'].isnull()) & (df_sf['Middle_name__c'].isnull()) & (df_sf['lastname'].isnull())])
df_sf = df_sf[(df_sf['firstname'].notnull()) & (df_sf['Middle_name__c'].notnull()) & (df_sf['lastname'].notnull())]

display(df_sf.head())



,Salesforce_org_ID,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash,is_email_commerical,domain_by_email,domain_by_profile,domain_by_profile_infer,domain_final
0,NaN,998,hubrepo hubrepo,hubrepo,0,nkissebe@gmail.com,0,2014-11-13 21:09:09,NaT,NaN,None,NaN,gmail.com,None,None,True,NaN,NaN,NaN,None
1,a0r5w00000V42cCAAR,1683,nanoHUB support,support,0,support@nanohub.org,0,2008-11-19 22:51:04,2008-11-19 23:55:30,university,purdue university,NaN,nanohub.org,"{university, purdue}",purdue-university,False,NaN,purdue.edu,purdue.edu,purdue.edu
2,a0r5w00000V42cCAAR,1684,Grid Statistics,gridstat,0,gridstat@nanohub.org,0,2008-11-18 17:29:56,2020-02-14 18:50:14,universitystaff,purdue university,NaN,nanohub.org,"{university, purdue}",purdue-university,False,NaN,purdue.edu,purdue.edu,purdue.edu
3,a0r5w00000V42cCAAR,1685,NCN NCN,ncn,0,ncn@nanohub.org,0,2008-11-11 19:17:04,NaT,NaN,purdue university,NaN,nanohub.org,"{university, purdue}",purdue-university,False,NaN,purdue.edu,purdue.edu,purdue.edu
4,a0r5w00000V42cCAAR,1686,nanoHUB nanoHUB,nanohub,0,apps@nanohub.org,0,2014-06-26 19:38:57,NaT,NaN,purdue university,NaN,nanohub.org,"{university, purdue}",purdue-university,False,NaN,purdue.edu,purdue.edu,purdue.edu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276205,NaN,345791,VENKATA ANIRUDH KOMAL KODAVALI,anirudh_k,0,venkata_a@ec.iitr.ac.in,0,2021-11-12 18:06:19,2021-11-12 18:06:19,government,iit roorkee,NaN,ec.iitr.ac.in,"{roorkee, iit}",iit-roorkee,False,NaN,NaN,None,None
276206,a0r5w00000V42qYAAR,345792,vishesh chandrikapure,visheshc38,0,visheshc38@gmail.com,0,2021-11-12 18:15:28,2021-11-12 18:15:28,universityundergraduate,wright state university,NaN,gmail.com,"{wright, state, university}",state-university-wright,True,NaN,wright.edu,wright.edu,wright.edu
276207,a0r5w00000V43O1AAJ,345793,Rahul Kumar,rahul20150133775,0,rahul20150133775@gmail.com,0,2021-11-12 18:30:07,2021-11-12 18:30:07,universityundergraduate,"indian institute of technology, roorkee",NaN,gmail.com,"{technology, institute, roorkee, indian}",indian-institute-roorkee-technology,True,NaN,iitr.ac.in,None,iitr.ac.in
276208,NaN,345795,Karan Kannoujiya,kannoujiya432,0,kannoujiya432@gmail.com,0,2021-11-12 18:37:49,2021-11-12 18:37:49,universityundergraduate,iit roorkee,NaN,gmail.com,"{roorkee, iit}",iit-roorkee,True,NaN,NaN,None,None


,firstname,Middle_name__c,lastname
125087,None,None,None


,firstname,Middle_name__c,lastname
0,hubrepo,,hubrepo
1,nanoHUB,,support
2,Grid,,Statistics
3,NCN,,NCN
4,nanoHUB,,nanoHUB


In [37]:
df_sf['nanoHUB_user_ID__c']            = df['id']
df_sf['nanoHUB_username__c']           = df['username']
df_sf['Email']                         = df['email'].fillna('').apply(lambda x: '' if '@invalid' in x else x).apply(lambda x: '' if '@' not in x else x)

# for sendEmail: 0 = opt-out, 1 = receive email. For salesforce HasOptedOutOfEmail, it's exact opposite
df_sf['HasOptedOutOfEmail']            = df['sendEmail'].apply(lambda x: 0 if x==1 else 1)
df_sf['nanoHUB_account_BLOCKED__c']    = df['block'].fillna(0)

# solidify time-related columns from datetime to string
df_sf['nanoHUB_registration_date__c']  = df['registerDate'].dt.date.fillna('').astype('str')
df_sf['nanoHUB_last_active_date__c']   = df['lastvisitDate'].dt.date.fillna('').astype('str')

# Tableau detailed view
df_sf['Detailed_user_timeline_to_Tableau__c'] = df_sf['Email'].apply(lambda x: 'https://tableauqa.itap.purdue.edu/views/profile/ProfileTimeline?Id%20Email='+x+'\
&:iframeSizedToWindow=true&:embed=y&:showAppBanner=false\
&:display_count=no&:showVizHome=no#6' if x != '' else '')

# nanoHUB user profile
df_sf['nanoHUB_user_page__c'] = df_sf['nanoHUB_user_ID__c'].apply(lambda x: 'https://nanohub.org/members/%d'%x if x != '' else '')
df_sf['Organization__c'] = df['organization'].fillna('')

df_sf['ORCID__c'] = df['orcid'].fillna('')

# derived information
df_sf['Organization_email_derived__c'] = df['Salesforce_org_ID'].fillna(' ')

sf_original_fields = df_sf.columns

# display
df_sf.head()
df_sf.tail()

,firstname,Middle_name__c,lastname,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c,Detailed_user_timeline_to_Tableau__c,nanoHUB_user_page__c,Organization__c,ORCID__c,Organization_email_derived__c
276205,VENKATA,ANIRUDH KOMAL,KODAVALI,345791,anirudh_k,venkata_a@ec.iitr.ac.in,1,0,2021-11-12,2021-11-12,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/345791,iit roorkee,,
276206,vishesh,,chandrikapure,345792,visheshc38,visheshc38@gmail.com,1,0,2021-11-12,2021-11-12,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/345792,wright state university,,a0r5w00000V42qYAAR
276207,Rahul,,Kumar,345793,rahul20150133775,rahul20150133775@gmail.com,1,0,2021-11-12,2021-11-12,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/345793,"indian institute of technology, roorkee",,a0r5w00000V43O1AAJ
276208,Karan,,Kannoujiya,345795,kannoujiya432,kannoujiya432@gmail.com,1,0,2021-11-12,2021-11-12,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/345795,iit roorkee,,
276209,sarana,solusindo,informatika,345796,solusitraining,annurulizza3@gmail.com,1,0,2021-11-12,2021-11-12,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/345796,harvard university,,a0r5w00000V42kEAAR


## To Salesforce Sales Cloud CRM

In [38]:

db_s.object_id = object_id
db_s.external_id = external_id

In [39]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000b4jN6AAI
hello
[Success] CSV upload successful. Job ID = 7505w00000b4jN6AAI
[Success] Closing job successful. Job ID = 7505w00000b4jN6AAI


In [40]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000b4jN6AAI',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-11-12T18:54:43.000+0000',
 'systemModstamp': '2021-11-12T18:55:50.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [41]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''
